# Les grandes étapes d'un projet de machine learning

## 1. Formaliser le problème

Lorsqu'on est confronté à un problème, il est souvent bénéfique d'y réfléchir à tête reposée et de l'analyser plutôt que de se lancer tête baissée dans le code. C'est particulièrement vrai en machine learning, puisqu'il est essentiel de _formaliser_ un problème pour qu'une machine puisse apprendre à le résoudre. Voici donc quelques questions à se poser systématiquement :

1. Le machine learning est-il réellement approprié pour mon problème ?
2. Quelles données vais-je pouvoir utiliser ?
3. Comment vais-je mesurer la performance de mon modèle ?

Pour ce TP, notre problème sera de différencier entre mines et rochers à partir de mesures prises par un sonar. Essayons de répondre aux trois questions ci-dessus :

_À compléter_

## 2. Traiter les données

### 2.a) Collecte des données

À l'aide de la fonction `read_csv` de la librairie `pandas`, importez les données contenues dans le fichier `sonar.all-data.csv` sous forme de DataFrame. Remarquez que les colonnes n'ont pas de noms dans le csv, il faut donc passer un paramètre `names` à la fonction `read_csv` : les 60 premières colonnes s'appelleront _F1, F2, ..., F60_ et la dernière colonne s'appelera _Type_


In [12]:
# À compléter

import pandas as pd

column_names = ['F' + str(i) for i in range(1, 61)] + ['Type']
# column_names = ['F{}'.format(i) for i in range(1, 61)] + ['Type']

df = pd.read_csv('sonar.all-data.csv', names=column_names)



Modifiez le Dataframe pour faire de la dernière colonne une caractéristique booléenne : remplacez les "M" par des 1 et les "R" par des 0.

In [13]:
# À compléter
df['Type'] = df['Type'].replace({"M":1, 'R':0})




Manque-t-il des données ? Si oui, remplacez les données manquantes par la valeur moyenne de la colonne correspondante.

Combien y a-t-il d'observations correspondant à des mines ? À des rochers ? Que peut-on en conclure ?

Mélangez les observations du dataframe.

In [14]:
# Vérifier s'il manque des données et remplacer par la valeur moyenne de la colonne
missing_data = df.isna().sum()
df = df.fillna(df.mean())

# Compter le nombre d'observations correspondant à des mines (1) et à des rochers (0)
nb_mines = (df['Type'] == 1).sum()
nb_rochers = (df['Type'] == 0).sum()

# Afficher le nombre d'observations de mines et de rochers
print("Nombre d'observations de mines (1) :", nb_mines)
print("Nombre d'observations de rochers (0) :", nb_rochers)

# Mélanger les observations du DataFrame
df = df.sample(frac=1).reset_index(drop=True)

Nombre d'observations de mines (1) : 111
Nombre d'observations de rochers (0) : 97



Pour que notre algorithme apprenne, on va séparer les observations du sonar de l'étiquette Mine ou Rocher. Séparez le dataframe en deux : un dataframe nommé `X` qui contient les 60 premières colonnes, et un dataframe nommé `Y` qui ne contient que la dernière colonne.

In [15]:

# Dernière colonne dans le DataFrame Y
Y = df.iloc[:, -1]


# X le reste sauf la dernier colonne
X = df.iloc[:, :-1]
# Avec les noms de collonnes
# X = df[["F"+str(i) for i in range(1,61)]]



### 2.b) Jeux d'entraînement, de validation et de test

Une fois nos données collectées et pré-traitées, il faut penser dès maintenant à les séparer en deux (ou trois) sous-ensembles pour avoir la possibilité d'évaluer notre solution.

À quoi correspondent ces sous-ensembles ? Pour le comprendre, imaginons que notre programme informatique est un étudiant qui révise pour un examen. Dans cette métaphore, l'étudiant doit apprendre en lisant son manuel de cours. À la fin de l'année, il a un examen qui consiste en plusieurs questions sur le cours. Pour que l'examen évalue bien la compréhension de l'étudiant, il est important que celui-ci n'ait jamais vu les questions auparavant (sinon il aurait pu bêtement mémoriser toutes les réponses sans réellement les comprendre).

Pour en revenir à notre algorithme de machine learning, on sépare donc les données à notre disposition en deux jeux :

- Un jeu d'entraînement, composé en général de 80% des observations à notre disposition, et qui correspond au manuel de cours.
- Un jeu de test (ou jeu d'évaluation), composé des 20% d'observations restantes, et qui correspond à l'examen final.

Il est primordial de ne pas utiliser les données de test avant l'évaluation à la toute fin du projet.

Séparez le dataframe `X` en deux moitiés `X_train` (70% des observations) et `X_test` (les 30% restants). Faites de même pour `Y` : attention, assurez-vous que les observations dans `Y_train` correspondent bien à celles dans `X_train`, et celles dans `Y_test` à `X_test`.

In [27]:
nb_observations_train = int(0.7*len(df)) 
nb_observations_test = len(df) - nb_observations_train
      
# Avec les indices
X_train = X.iloc[nb_observations_train:, :]
X_test = X.iloc[:nb_observations_train, :]



Y_train = Y.iloc[nb_observations_train:]
Y_test = Y.iloc[:nb_observations_train]



## 3. Entraîner un modèle

### 3.a) Qu'est-ce qu'un modèle ? Considérations générales

Un modèle est une représentation plus ou moins précise de la réalité. On l'appelle aussi une hypothèse.

Par exemple, si on est intéressé par le lien entre la taille des mains et des pieds, on peut formuler l'hypothèse que ces deux grandeurs sont liées par la relation&nbsp;:&nbsp;$taille~des~pieds~=~3~\times~taille~des~mains~+~2$. On a modélisé la relation entre les deux grandeurs par une fonction linéaire, ce modèle appartient donc à la _classe de modèles_ des fonctions linéaires.

Un problème de machine learning consiste à chercher le meilleur modèle possible dans une certaine classe de modèles. En théorie, on pourrait prendre comme classe de modèles l'ensemble de tous les programmes existant : on serait alors sûr que le meilleur programme possible se trouverait dans l'ensemble. En pratique, ce serait beaucoup trop coûteux en temps et en énergie, on se limite donc à des classes de modèles plus simples à explorer.
<!---
Pour plus tard : choisir des hyperparamètres, valider ses choix
--->
### 3.b) Une première classe de modèles pour classifier : les "$k$-plus proches voisins"

Une manière très simple d'apprendre à classifier à partir d'observations consiste à comparer entre elles les observations qui se ressemblent. Par exemple : on me donne une photo d'animal à classifier, donc je la compare aux 20 photos d'animaux dans ma collection qui lui ressemblent le plus. Parmi ces photos, il y a 14 photos de chat, 5 photos de tigre et 1 photo d'alligator. J'en conclue que la photo que l'on m'a donné est une photo de chat.

Cet algorithme élémentaire s'appelle "$k$-nearest-neighbors" : on regarde ce que font les $k$ (en l'occurence $20$) plus proches voisins pour classifier.

Je n'ai pas été tout à fait honnête et j'ai omis de mentionner un point potentiellement complexe dans cet algorithme apparemment simpliste. Pouvez-vous voir quelle partie de ma description mériterait plus d'explications ? (Indice : imaginez que vous implémentez en détail cet algorithme, il y a un moment où vous risquez de coincer...)

_À compléter_

### 3.c) Scikit-learn

La librairie `scikit-learn` (souvent abrégé `sklearn`) implémente un [grand nombre](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) d'algorithmes "traditionnels" de machine learning (presque tous les algorithmes à l'exception des algorithmes de deep learning).

Si ce n'est pas déjà fait, installez `scikit-learn` sur votre machine :

In [17]:
!pip install scikit-learn

zsh:1: command not found: pip


Toutes les classes de modèles proposées par `sklearn` ont trois fonctions essentielles :

1. `fit` prend en paramètre des données d'entraînements et effectue l'apprentissage d'un modèle dans la classe à partir de ces données.
2. `score` prend en paramètre des données de test et renvoit une mesure de la performance du modèle sur ces données.
3. `predict` prend en paramètre une observation et renvoit la prédiction du modèle entraîné.

Si vous maîtrisez ces trois fonctions, félicitations, vous maîtrisez (presque) `sklearn` !

Créez un modèle de type `KNeighborsClassifier` à l'aide de `scikit-learn`.

In [28]:
from sklearn.neighbors import KNeighborsClassifier

cls = KNeighborsClassifier()


Entraînez le modèle avec les données d'entraînement.

In [29]:
cls.fit(X_train,Y_train)

KNeighborsClassifier()

## 4. Tester le modèle final

Une fois le modèle entraîné, il est important de vérifier qu'il fonctionne "dans la vraie vie", c'est-à-dire avec des données qu'il n'a jamais vu auparavant.

Utilisez la fonction `score` pour evaluer le modèle entraîné.

In [30]:
cls.score(X_train, Y_train)

0.7619047619047619

In [31]:
# Exemple de prédiction

donnes_sous_marin = X_test.iloc[0, :]
cls.predict([donnes_sous_marin])

/Users/vincentchabran/Documents/gitHub/masterAflokkat/machineLearning/cours1-30:09:23/.venv/lib/python3.12/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1])

Vous avez presque terminé votre premier projet de machine learning, mais votre travail ne s'arrête pas là ! Une fois un modèle entraîné et testé, il faut souvent le "vendre" aux clients. Cela signifie principalement expliquer vos choix et leur pertinence, pour que le client ait confiance en votre IA. Pour ce TP guidé, vous n'avez pas eu beaucoup de choix à faire, donc je vous épargne cette tâche. Cependant, le TP suivant sur la validation va vous entraîner sur le sujet !

In [ ]:
# Pour enregistrer le model

import pickle

with open("modele.pkl", "wb") as f :
   pickle.dump(cls, f)
   f.close()


In [ ]:
with open("modele.pkl", "wb") as f :
   mon_cls = pickle.load(cls, f)
   f.close()

mon_cls.score(X_test, Y_test)